In [ ]:
!pip install xarray
!pip install ipytree

In [1]:
from dask.distributed import Client, progress, LocalCluster
from rechunker import rechunk
import s3fs
import xarray as xr
import zarr
import dask.array as dsa
import shutil
from dask.diagnostics import ProgressBar

## Start the Dask Cluster

In [2]:
cluster = LocalCluster(n_workers=20)
client = Client(cluster)
client
# client.close()
# cluster.close()

Client Scheduler: tcp://127.0.0.1:37539 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 20 Cores: 80 Memory: 524.29 GB


## Load the MUR SST Dataset from AWS PDS S3

In [3]:
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-west-2'))
s3_store = s3fs.S3Map(root='mur-sst/zarr', s3=s3, check=False)
ds_zarr = zarr.open_consolidated(s3_store, mode='r')

In [42]:
%%time
#ds = xr.open_zarr(s3_store, consolidated=True)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [17]:
print(zarr.tree(ds_zarr))

/
 ├── analysed_sst (6443, 17999, 36000) int16
 ├── analysis_error (6443, 17999, 36000) int16
 ├── lat (17999,) float32
 ├── lon (36000,) float32
 ├── mask (6443, 17999, 36000) int8
 ├── sea_ice_fraction (6443, 17999, 36000) int8
 └── time (6443,) int64


## Rechunker Plan

In [40]:
s3 = s3fs.S3FileSystem(client_kwargs=dict(region_name='us-west-2'))
s3_rechunk_store = s3fs.S3Map(root='ds-projects/eodc/mursst/6443100100-group.zarr', create=True, s3=s3)
s3_tmp_store = s3fs.S3Map(root='ds-projects/eodc/mursst/tmp/tmp-group.zarr', create=True, s3=s3)

In [41]:
target_chunks = {
    'analysed_sst': {'time': 6443, 'lat': 100, 'lon': 100},
    'lat': None,
    'lon': None,
    'time': None
}
max_mem = '2GB'

array_plan = rechunk(ds_zarr, target_chunks, max_mem, s3_rechunk_store, s3_tmp_store)
array_plan

AssertionError: 

In [ ]:
with ProgressBar():
    array_plan.execute()